# 43008: Reinforcement Learning

## Week 11: Part A: Exploring Monte Carlo Tree Search (MCTS)

This notebook will explore the implementation of a MCTS method using MCTS-simple library on Tic-Tac-Toe and OpenAI Gym Environment.


### What you will learn?
* Using MCTS
* Train on a Tic-Tac-Toe environment
* Play with the trained agent

---


### Install the library

In [1]:
!sudo apt-get update
# Updating the package list and installing necessary packages for the project
!sudo apt-get install -y cmake
# Updating the package list and installing ffmpeg and freeglut3-dev for visualization, xvfb for virtual display
!sudo apt-get install -y ffmpeg freeglut3-dev xvfb

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [2]:
# Installing the Box2D environment for physics simulation
!pip install gymnasium
!pip install swig
!pip install gymnasium[box2d]
!pip install mcts-simple

### Create the Tic-Tac-Toe environment

In [3]:
from mcts_simple import *

class TicTacToe(Game):
    def __init__(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.players = ["X", "O"]
    ''' Try to adapt this rendering
    def render1(self):
              res = []
        for i in range(self.board_size):
            if i == 0:
                res.append('┌───┬───┬───┐')
            elif i in [1, 2]:
                res.append('├───┼───┼───┤')

            out = '│ '
            for j in range(self.board_size):
                token = self.getMarker(self.board[i, j])
                out += token + ' │ '
            res.append(out)
        res.append('└───┴───┴───┘')
        return '\n'.join(res)
    '''

    def render(self):
        board = ""
        board += "|".join(self.board[0]) + "\n"
        board += "─────\n"
        board += "|".join(self.board[1]) + "\n"
        board += "─────\n"
        board += "|".join(self.board[2]) + "\n"
        print(board)

    def get_state(self):
        return [(self.board[row][col] == self.players[0]) - (self.board[row][col] == self.players[1]) for row in range(len(self.board)) for col in range(len(self.board[row]))]

    def number_of_players(self):
        return len(self.players)

    def current_player(self):
        return int(self.players[0] == "X")

    def possible_actions(self):
        return [row * 3 + col for row in range(len(self.board)) for col in range(len(self.board[row])) if self.board[row][col] == " "]

    def take_action(self, action):
        self.board[action // 3][action % 3] = self.players[0]
        self.players.append(self.players.pop(0))

    def has_outcome(self):
        result = False
        # check horizontal
        result |= self.board[0][0] == self.board[0][1] and self.board[0][0] == self.board[0][2] and self.board[0][0] != " "
        result |= self.board[1][0] == self.board[1][1] and self.board[1][0] == self.board[1][2] and self.board[1][0] != " "
        result |= self.board[2][0] == self.board[2][1] and self.board[2][0] == self.board[2][2] and self.board[2][0] != " "
        # check vertical
        result |= self.board[0][0] == self.board[1][0] and self.board[0][0] == self.board[2][0] and self.board[0][0] != " "
        result |= self.board[0][1] == self.board[1][1] and self.board[0][1] == self.board[2][1] and self.board[0][1] != " "
        result |= self.board[0][2] == self.board[1][2] and self.board[0][2] == self.board[2][2] and self.board[0][2] != " "
        # check diagonal
        result |= self.board[0][0] == self.board[1][1] and self.board[0][0] == self.board[2][2] and self.board[0][0] != " "
        result |= self.board[0][2] == self.board[1][1] and self.board[0][2] == self.board[2][0] and self.board[0][2] != " "
        return result or not any(" " in space for space in self.board)

    def winner(self):
        winners = []
        # check horizontal
        winners += [self.board[0][0] == "X"] if self.board[0][0] == self.board[0][1] and self.board[0][0] == self.board[0][2] and self.board[0][0] != " " else []
        winners += [self.board[1][0] == "X"] if self.board[1][0] == self.board[1][1] and self.board[1][0] == self.board[1][2] and self.board[1][0] != " " else []
        winners += [self.board[2][0] == "X"] if self.board[2][0] == self.board[2][1] and self.board[2][0] == self.board[2][2] and self.board[2][0] != " " else []
        # check vertical
        winners += [self.board[0][0] == "X"] if self.board[0][0] == self.board[1][0] and self.board[0][0] == self.board[2][0] and self.board[0][0] != " " else []
        winners += [self.board[0][1] == "X"] if self.board[0][1] == self.board[1][1] and self.board[0][1] == self.board[2][1] and self.board[0][1] != " " else []
        winners += [self.board[0][2] == "X"] if self.board[0][2] == self.board[1][2] and self.board[0][2] == self.board[2][2] and self.board[0][2] != " " else []
        # check diagonal
        winners += [self.board[0][0] == "X"] if self.board[0][0] == self.board[1][1] and self.board[0][0] == self.board[2][2] and self.board[0][0] != " " else []
        winners += [self.board[0][2] == "X"] if self.board[0][2] == self.board[1][1] and self.board[0][2] == self.board[2][0] and self.board[0][2] != " " else []
        # check draw
        if len(winners) == 0 and not any(" " in space for space in self.board):
            winners = [player == "X" for player in self.players]
        return winners


### Create the game instance and start training

In [4]:
game = TicTacToe()

In [5]:
# tree = UCT(game, allow_transpositions = True, training = True)
tree = MCTS(game, allow_transpositions = True, training = True)
tree.self_play(iterations = 10000)

Training:   0%|          | 0/10000 [00:00<?, ?it/s]

### Check self play result

In [6]:
tree.training = False
tree.self_play()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

 | | 
─────
 | | 
─────
 | | 

 | | 
─────
 | | 
─────
 |X| 

O| | 
─────
 | | 
─────
 |X| 

O| |X
─────
 | | 
─────
 |X| 

O|O|X
─────
 | | 
─────
 |X| 

O|O|X
─────
 | | 
─────
 |X|X

O|O|X
─────
O| | 
─────
 |X|X

O|O|X
─────
O| | 
─────
X|X|X



### Save the trained MCTS model

In [7]:
tree.save("tictactoe.mcts")

### Play with the trained agent!

In [8]:
pwd

'/home/sagemaker-user/RL_sagemaker/RL_week11'

In [9]:
import random
from copy import deepcopy

human_player = 1 # X

game = TicTacToe()
# tree = UCT(game, allow_transpositions = True, training = False)
tree = MCTS(game, allow_transpositions = True, training = False)
tree.load("tictactoe.mcts")

node = tree.root
while not game.has_outcome():
    game.render()
    actions = game.possible_actions()
    if game.current_player() == human_player:
        print("Possible actions:", actions)
        action = int(input("> "))
        assert action in actions
        if node is not None and action in node.children:
            node = node.children[action]
        else:
            node = None
    else:
        if node is not None and len(node.children) > 0:
            action = node.choose_best_action(tree.training)
            node = node.children[action]
        else:
            action = random.choice(actions)
            node = None
    game.take_action(action)
game.render()

 | | 
─────
 | | 
─────
 | | 

Possible actions: [0, 1, 2, 3, 4, 5, 6, 7, 8]


>  4


 | | 
─────
 |X| 
─────
 | | 

 | | 
─────
 |X|O
─────
 | | 

Possible actions: [0, 1, 2, 3, 6, 7, 8]


>  2


 | |X
─────
 |X|O
─────
 | | 

O| |X
─────
 |X|O
─────
 | | 

Possible actions: [1, 3, 6, 7, 8]


...

O| |X
─────
X|X|O
─────
 | | 

O| |X
─────
X|X|O
─────
O| | 

Possible actions: [1, 7, 8]


>  7


O| |X
─────
X|X|O
─────
O|X| 

O|O|X
─────
X|X|O
─────
O|X| 

Possible actions: [8]


>  8


O|O|X
─────
X|X|O
─────
O|X|X



## Use a Gym environment to train with MCTS - Cartpole

In [17]:
import imageio
import base64
import IPython
import gymnasium as gym
from mcts_simple import *

class CartPole(Game):
    """
    The episode ends if any one of the following occurs:
        * Termination: Pole Angle is greater than ±12°
        * Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
        * Truncation: Episode length is greater than 500 (200 for v0)
    """
    def __init__(self):
        self.env = gym.make("CartPole-v1", render_mode = "rgb_array")
        self.current_state, _ = self.env.reset()
        self.frames = []
        self.terminated, self.truncated = False, False

    def render(self):
        self.frames.append(self.env.render())
        if self.has_outcome():
            IPython.display.display(IPython.display.HTML(data = f"""
            <video controls src = "data:video/mp4;base64,{base64.b64encode(imageio.mimsave(
            "<bytes>", self.frames, "MP4", fps = 20, **{"macro_block_size": None})).decode()}"></video>
            """))
            self.frames.clear()

    def get_state(self):
        return self.current_state

    def number_of_players(self):
        return 1

    def current_player(self):
        return 0

    def possible_actions(self):
        return [i for i in range(self.env.action_space.n)]

    def take_action(self, action):
        if not self.has_outcome():
            self.current_state, _, self.terminated, self.truncated, _ = self.env.step(action)

    def has_outcome(self):
        return self.terminated or self.truncated

    def winner(self):
        # Noting that backprop code is: node.w += (prev_node.player in winners) / number_of_winners
        # It is possible to manipulate list size = self.env._max_episode_steps - self.env._elapsed_steps, since there will always be only 1 player
        # winner() will return a reward instead, where 0 <= reward <= 1, where it will increase exponentially as elapsed steps increase
        return [self.current_player() for _ in range(max(1, self.env._max_episode_steps - self.env._elapsed_steps + 1))]


### Create Cartpole instance and start training

In [18]:
game = CartPole()

In [19]:
tree = OpenLoopMCTS(game, training = True)
tree.self_play(iterations = 1000) # try 10000

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

### Check test outcome

In [20]:
!pip install imageio-ffmpeg

In [21]:
tree.training = False
tree.self_play()


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5205:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5205:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5205:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5728:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM default


### Save the trained model

In [15]:
tree.save("cartpole.mcts")

# TASK: Try a different Gym environment and train and Test on MCTS

In [16]:
### WRITE YOUR CODE HERE ###